In [ ]:
athena_db = 'trivago'
s3_bucket = ''
date = '2019-08-01'

In [ ]:
import awswrangler
session = awswrangler.Session()

In [ ]:
import pandas

# Get USPs per accommodation and add destination_id

In [ ]:
def get_destination_ids(accommodation_ids):
    with open('query.sql') as f:
        df = session.pandas.read_sql_athena(
            sql=f.read().format(date=repr(date)),
            database=athena_db
        )
        
    return df[df.accommodation_id.isin(accommodation_ids)]

In [ ]:
def get_filtered_usps():
    df = session.pandas.read_sql_athena(
        sql="SELECT * FROM unique_selling_points WHERE date = {date}".format(date=repr(date)),
        database=athena_db
    )
    
    return df

In [ ]:
usp_df = get_filtered_usps()
destination_ids = get_destination_ids(usp_df["accommodation_id"].values)
usp_df = usp_df.merge(destination_ids, how="left", on="accommodation_id")

In [ ]:
usp_df.sample(5)

In [ ]:
usp_df.groupby("usp")['destination_id'].nunique().sort_values(ascending=False)

In [ ]:
session.pandas.to_parquet(
    dataframe=usp_df,
    path=f"s3://{s3_bucket}/usp/output",
    partition_cols=["date"],
)